In [1]:
from navigation.navigator import Navigator

from utils import print_models, model_as_str, assumptions_as_str

In [2]:
nav = Navigator()
nav.load("bike.lp")

# Use case 1: identifying desired features

In [3]:
print_models(nav.compute_diverse_models(3))

Model 1: {color(red), framesize(l), type(race), weight(5)}
Model 2: {bagsize(1,m), bagsize(2,l), bagsize(3,l), bagsize(4,l), color(green), framesize(s), include(bell), include(light), include(mudguards), type(travel), weight(17)}
Model 3: {bagsize(1,s), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(blue), framesize(m), include(light), type(city), weight(12)}


In [4]:
nav.add_assumption("framesize(l)", True)
nav.add_assumption("include(light)", True)
nav.add_assumption("color(blue)", True)
print_models(nav.compute_diverse_models(3))

Model 1: {color(blue), framesize(l), include(light), type(city), weight(6)}
Model 2: {bagsize(1,m), bagsize(2,l), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(bell), include(light), include(mudguards), type(travel), weight(19)}
Model 3: {bagsize(1,s), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(blue), framesize(l), include(light), type(race), weight(13)}


In [5]:
nav.add_assumption("type(race)", False)
models = nav.compute_diverse_models(3)
print_models(models)

Model 1: {color(blue), framesize(l), include(light), type(city), weight(6)}
Model 2: {bagsize(1,m), bagsize(2,l), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(bell), include(light), include(mudguards), type(travel), weight(19)}
Model 3: {bagsize(1,s), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(blue), framesize(l), include(light), type(city), weight(13)}


In [6]:
print(assumptions_as_str(nav.get_assumptions()))

{(color(blue), True), (framesize(l), True), (include(light), True), (type(race), False)}


# Use case 2: improving a configuration

In [7]:
start_config = models[1]
print(model_as_str(start_config))

{bagsize(1,m), bagsize(2,l), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(bell), include(light), include(mudguards), type(travel), weight(19)}


In [8]:
nav.add_assumption("bagsize(4,l)", True)
nav.add_assumption("include(mudguards)", False)
nav.add_assumption("bagsize(2,l)", False)
print_models(nav.compute_similar_models(3, [start_config]))

Model 1: {bagsize(1,m), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(bell), include(light), type(travel), weight(15)}
Model 2: {bagsize(1,m), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(bell), include(light), type(city), weight(15)}
Model 3: {bagsize(1,m), bagsize(3,l), bagsize(4,l), color(blue), framesize(l), include(light), type(travel), weight(14)}


# Use case 3: possible options and effects of assumptions

In [9]:
nav.clear_assumptions()

In [10]:
cautious = nav.compute_cautious_consequences()
print(model_as_str(cautious))

{}


In [11]:
facets = nav.compute_facets()
print(model_as_str(facets))

{bagsize(1,m), bagsize(1,s), bagsize(2,l), bagsize(2,m), bagsize(2,s), bagsize(3,l), bagsize(3,m), bagsize(3,s), bagsize(4,l), bagsize(4,m), color(blue), color(green), color(red), framesize(l), framesize(m), framesize(s), include(bell), include(light), include(mudguards), type(city), type(race), type(travel), weight(10), weight(11), weight(12), weight(13), weight(14), weight(15), weight(16), weight(17), weight(18), weight(19), weight(3), weight(4), weight(5), weight(6), weight(7), weight(8), weight(9)}


In [12]:
weights = set(filter(lambda x: x.name == "weight", facets))
print(model_as_str(weights))

{weight(10), weight(11), weight(12), weight(13), weight(14), weight(15), weight(16), weight(17), weight(18), weight(19), weight(3), weight(4), weight(5), weight(6), weight(7), weight(8), weight(9)}


In [13]:
nav.add_assumption("type(race)", True)
new_cautious = nav.compute_cautious_consequences()
print(f"new cautious consequences by assuming type(race): {model_as_str(new_cautious - cautious)}")
new_facets = nav.compute_facets()
print(f"removed facets by assuming type(race): {model_as_str(facets - new_facets)}")

new cautious consequences by assuming type(race): {type(race)}
removed facets by assuming type(race): {bagsize(2,l), bagsize(3,l), bagsize(4,l), include(mudguards), type(city), type(race), type(travel), weight(16), weight(17), weight(18), weight(19)}


In [14]:
nav.remove_assumption("type(race)", True)
nav.add_assumption("type(city)", True)
new_cautious = nav.compute_cautious_consequences()
print(f"new cautious consequences by assuming type(city): {model_as_str(new_cautious - cautious)}")
new_facets = nav.compute_facets()
print(f"removed facets by assuming type(city): {model_as_str(facets - new_facets)}")

new cautious consequences by assuming type(city): {include(light), type(city)}
removed facets by assuming type(city): {include(light), type(city), type(race), type(travel), weight(3)}


# Use case 4: generalised assumptions

In [15]:
nav.clear_assumptions()

In [16]:
nav.add_constraint(":- bagsize(X,l).")
print_models(nav.compute_diverse_models(3))

Model 1: {bagsize(1,m), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(blue), framesize(l), include(bell), type(race), weight(14)}
Model 2: {bagsize(1,s), bagsize(2,s), bagsize(3,s), color(green), framesize(m), include(light), include(mudguards), type(city), weight(9)}
Model 3: {bagsize(1,s), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(red), framesize(s), include(bell), include(light), include(mudguards), type(city), weight(13)}


In [17]:
nav.get_rules()

{':- bagsize(X,l).': True}

In [18]:
nav.deactivate_rule(":- bagsize(X,l).")
print_models(nav.compute_diverse_models(3))

Model 1: {bagsize(2,l), bagsize(3,l), bagsize(4,l), color(red), framesize(m), include(light), type(city), weight(14)}
Model 2: {bagsize(1,m), bagsize(2,m), bagsize(3,m), bagsize(4,m), color(green), framesize(s), include(bell), type(race), weight(12)}
Model 3: {bagsize(1,s), bagsize(2,s), bagsize(3,s), bagsize(4,l), color(blue), framesize(l), include(light), include(mudguards), type(travel), weight(13)}


In [19]:
nav.add_constraint(":- weight(X), X >= 10.")
print_models(nav.compute_diverse_models(3))

Model 1: {bagsize(2,l), bagsize(3,l), color(red), framesize(s), type(travel), weight(9)}
Model 2: {bagsize(1,m), bagsize(4,l), color(green), framesize(m), type(travel), weight(9)}
Model 3: {bagsize(2,m), bagsize(3,m), bagsize(4,m), color(blue), framesize(s), type(race), weight(9)}


In [20]:
nav.deactivate_rule(":- weight(X), X >= 10.")
nav.get_rules()

{':- bagsize(X,l).': False, ':- weight(X), X >= 10.': False}

# Use case 5: add optimization statements

In [21]:
nav.add_rule("#minimize{ X : weight(X) }.", permanent=True)
print_models(nav.compute_models(5))

Model 1: {color(blue), framesize(l), include(bell), type(race), weight(6)}
Model 2: {color(green), framesize(l), include(bell), type(race), weight(6)}
Model 3: {color(red), framesize(l), include(bell), type(race), weight(6)}
Model 4: {color(green), framesize(l), type(race), weight(5)}
Model 5: {color(blue), framesize(l), type(race), weight(5)}


In [22]:
nav.enable_optimization()
print_models(nav.compute_models(0))

Model 1: {color(green), framesize(s), type(race), weight(3)}
Model 2: {color(red), framesize(s), type(race), weight(3)}
Model 3: {color(blue), framesize(s), type(race), weight(3)}


In [23]:
nav.disable_optimization()

# Use case 6: user defined features

In [24]:
nav.add_rule("numSmallBags(C) :- C = #count{ X : bagsize(X,s) }.")
print_models(nav.compute_models())

Model 1: {bagsize(2,l), bagsize(3,l), bagsize(4,l), color(red), framesize(s), include(bell), include(light), numSmallBags(0), type(city), weight(14)}


In [25]:
nav.add_assumption("numSmallBags(2)", True)
print_models(nav.compute_diverse_models(3))

Model 1: {bagsize(1,s), bagsize(2,l), bagsize(3,s), bagsize(4,l), color(red), framesize(s), include(bell), include(mudguards), numSmallBags(2), type(travel), weight(13)}
Model 2: {bagsize(1,s), bagsize(2,s), bagsize(3,l), color(blue), framesize(l), include(bell), include(light), numSmallBags(2), type(city), weight(12)}
Model 3: {bagsize(1,s), bagsize(2,s), bagsize(3,m), bagsize(4,m), color(green), framesize(m), numSmallBags(2), type(race), weight(10)}


In [26]:
nav.clear_assumptions()
nav.add_constraint(":- numSmallBags(X), X < 3.")
print_models(nav.compute_diverse_models(3))

Model 1: {bagsize(1,s), bagsize(2,s), bagsize(3,s), bagsize(4,l), color(red), framesize(s), include(bell), include(light), include(mudguards), numSmallBags(3), type(city), weight(12)}
Model 2: {bagsize(1,s), bagsize(2,s), bagsize(3,s), bagsize(4,m), color(blue), framesize(l), numSmallBags(3), type(race), weight(10)}
Model 3: {bagsize(1,s), bagsize(2,s), bagsize(3,s), bagsize(4,l), color(green), framesize(m), include(light), numSmallBags(3), type(travel), weight(11)}


In [27]:
nav.add_assumption("numSmallBags(4)", True)
print_models(nav.compute_diverse_models(3))

UNSAT
